# Image Cropper

## Dependencies

In [49]:
import os
from PIL import Image
import shutil

## Parameters

In [50]:
input_root = "after_anonym"
output_root = "after_crop"

## Copy images in right folder

In [51]:
def MoveFiles(left_rgb_path, thermal_path, folder_name):
    # Prepare output folder
    out_folder = os.path.join(output_root, folder_name)
    os.makedirs(out_folder, exist_ok=True)
    
    # Copy left_rbg.png and thermal.png as is
    shutil.copy2(left_rgb_path, os.path.join(out_folder, "left_rgb.png"))
    shutil.copy2(thermal_path, os.path.join(out_folder, "thermal.png"))
    
    print(f"    Copied left_rgb.png and thermal.png to {out_folder}")
    return out_folder

## Load images

In [52]:
def LoadImages(left_rgb_path, thermal_path):
    left_rgb = Image.open(left_rgb_path)
    thermal = Image.open(thermal_path)
    return left_rgb, thermal

## Crop Images

### Version 1
The first version output an image of the same size of the thermal image. It focus on having two image of the same size that show approximately the same thing

In [53]:
def CropToSizeImage(left_rgb, thermal):
    # arbitrary offset values
    offset_x = 165
    offset_y = 47

    # Determine crop box
    thermal_width, thermal_height = thermal.size
    crop_left = offset_x
    crop_upper = offset_y
    crop_right = offset_x + thermal_width
    crop_lower = offset_y + thermal_height

    # Safety check: avoid cropping outside bounds
    left_width, left_height = left_rgb.size
    crop_right = min(crop_right, left_width)
    crop_lower = min(crop_lower, left_height)
    crop_box = (crop_left, crop_upper, crop_right, crop_lower)

    cropped_left_rgb = left_rgb.crop(crop_box) 
    return cropped_left_rgb

### Version 2
The second version output an image of a different size of the thermal image to adjust the scale of the two images. It focus on having two images that show objects the same size in the foreground despite having different size.

In [54]:
def CropAndScaleImage(left_rgb, thermal):
    # arbitrary offset values

    # keep same ratio as thermal image
    # offset_x = 194
    # offset_y = 62
    # scale = 257/320

    # change ratio to fit thermal image
    offset_x = 185
    offset_y = 62
    scale_x = 267/320
    scale_y = 196/240

    # Determine crop box
    thermal_width, thermal_height = thermal.size
    crop_left = offset_x
    crop_upper = offset_y
    crop_right = int(offset_x + thermal_width*scale_x)
    crop_lower = int(offset_y + thermal_height*scale_y)

    # Safety check: avoid cropping outside bounds
    left_width, left_height = left_rgb.size
    crop_right = min(crop_right, left_width)
    crop_lower = min(crop_lower, left_height)
    crop_box = (crop_left, crop_upper, crop_right, crop_lower)

    cropped_left_rgb = left_rgb.crop(crop_box) 
    return cropped_left_rgb

## Save Image


In [55]:
def SaveImage(cropped_left, cropped_and_scaled_left, out_folder):
    cropped_left.save(os.path.join(out_folder, "cropped_left_rgb.png"))
    cropped_and_scaled_left.save(os.path.join(out_folder, "cropped_and_scaled_left_rgb.png"))

    print(f"    Saved images to {out_folder}")


## Loop to all files in input folder

In [56]:
for folder_name in os.listdir(input_root):
    folder_path = os.path.join(input_root, folder_name)
    if not os.path.isdir(folder_path):
        continue  # Skip files
    
    print(f"    Processing folder: {folder_name}")

    left_rgb_path = os.path.join(folder_path, "left_rgb.png")
    thermal_path = os.path.join(folder_path, "thermal.png")

    out_folder = MoveFiles(left_rgb_path, thermal_path, folder_name)
    left_rgb, thermal = LoadImages(left_rgb_path, thermal_path)
    cropped_left_rgb = CropToSizeImage(left_rgb, thermal)
    cropped_and_scaled_left_rgb = CropAndScaleImage(left_rgb, thermal)

    SaveImage(cropped_left_rgb, cropped_and_scaled_left_rgb, out_folder)

print("Processing complete!")

    Processing folder: 2025-03-24_14-11-45-485544
    Copied left_rgb.png and thermal.png to after_crop\2025-03-24_14-11-45-485544
    Saved images to after_crop\2025-03-24_14-11-45-485544
    Processing folder: 2025-03-24_14-14-16-187470
    Copied left_rgb.png and thermal.png to after_crop\2025-03-24_14-14-16-187470
    Saved images to after_crop\2025-03-24_14-14-16-187470
    Processing folder: 2025-03-24_14-20-10-670750
    Copied left_rgb.png and thermal.png to after_crop\2025-03-24_14-20-10-670750
    Saved images to after_crop\2025-03-24_14-20-10-670750
    Processing folder: 2025-04-10_15-08-58-938683
    Copied left_rgb.png and thermal.png to after_crop\2025-04-10_15-08-58-938683
    Saved images to after_crop\2025-04-10_15-08-58-938683
Processing complete!
